Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import numpy as np

Ready data for linear regression

In [7]:
df = pd.read_pickle("Training_TestData.pkl")

# Drop customer id
df = df.drop(labels="Customer",axis=1)

In [8]:
df_backup = pd.read_pickle("Training_TestData.pkl")

One Hot encode stuff

In [9]:
from sklearn.preprocessing import OneHotEncoder

OneHotEncoder(
    categories='auto',  # Categories per feature
    drop=None, # Whether to drop one of the features
    sparse=True, # Will return sparse matrix if set True
    handle_unknown='error' # Whether to raise an error 
)    

OneHotEncoder()

In [10]:
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[["Product","Area","Season"]])

In [11]:
transformed.toarray()

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [12]:
ohe.categories_

[array(['Benzin', 'Motorin'], dtype=object),
 array(['Akdeniz Bölgesi', 'Doğu Anadolu Bölgesi', 'Ege Bölgesi',
        'Güneydoğu Anadolu Bölgesi', 'Karadeniz Bölgesi',
        'Marmara Bölgesi', 'İç Anadolu Bölgesi'], dtype=object),
 array(['Autumn', 'Fall', 'Summer', 'Winter'], dtype=object)]

In [13]:
df[ohe.categories_[0]] = transformed.toarray()[:,:2]
df[ohe.categories_[1]] = transformed.toarray()[:,2:9]
df[ohe.categories_[2]] = transformed.toarray()[:,9:]

In [14]:
# Drop encoded columns
df.drop(labels=["Area","Product","Season"],axis=1,inplace=True)

In [15]:
df

,WeekStarting,Litres,Price(t-1),NewCars,WeekendHolidays,WeekdayHolidays,GDPGrowth%,MoMInflation%,WoWUSDChange%,Litres(w-1),Litres(w-2),Litres(w-3),Litres(w-49),Litres(w-50),Litres(w-51),Litres(w-52),WeekNumber,Benzin,Motorin,Akdeniz Bölgesi,Doğu Anadolu Bölgesi,Ege Bölgesi,Güneydoğu Anadolu Bölgesi,Karadeniz Bölgesi,Marmara Bölgesi,İç Anadolu Bölgesi,Autumn,Fall,Summer,Winter
0,2014-01-06,4242.0,5.07,5015,0.0,0.0,0.044,0.01,-0.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2014-01-06,29017.0,4.40,5015,0.0,0.0,0.044,0.01,-0.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2014-01-06,4572.0,5.04,5015,0.0,0.0,0.044,0.01,-0.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2014-01-06,30681.0,4.34,5015,0.0,0.0,0.044,0.01,-0.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2014-01-06,25009.0,4.40,5015,0.0,0.0,0.044,0.01,-0.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587421,2019-12-02,NaN,7.14,2004,0.0,0.0,-0.015,2.00,0.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
587422,2019-12-09,NaN,7.14,2004,0.0,0.0,0.009,2.00,0.55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
587423,2019-12-16,NaN,7.14,2004,0.0,0.0,0.009,0.38,2.21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
587424,2019-12-23,NaN,7.14,2004,0.0,0.0,0.009,0.38,0.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Drop First Year

In [16]:
df = df[df["WeekStarting"] > "2015-01-01"]

In [17]:
df = df.reset_index(drop=True)

Standard Scaling

In [20]:
from sklearn.preprocessing import StandardScaler

X = df.drop(labels=["Litres","WeekStarting"],axis=1)
y = df["Litres"]

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [21]:
X_scaled

array([[-0.96191852, -0.86748063, -0.25804112, ..., -0.57981162,
        -0.57555289,  1.78731669],
       [-1.43218329, -0.86748063, -0.25804112, ..., -0.57981162,
        -0.57555289,  1.78731669],
       [-1.43218329, -0.86748063, -0.25804112, ..., -0.57981162,
        -0.57555289,  1.78731669],
       ...,
       [ 1.76322516, -1.0317202 , -0.25804112, ..., -0.57981162,
        -0.57555289,  1.78731669],
       [ 1.76322516, -1.0317202 , -0.25804112, ..., -0.57981162,
        -0.57555289,  1.78731669],
       [ 1.91070956, -1.02639381, -0.25804112, ..., -0.57981162,
        -0.57555289,  1.78731669]])

Assign scaled data to df

In [30]:
cols = df.columns
df[cols[2:]] = X_scaled
df

,WeekStarting,Litres,Price(t-1),NewCars,WeekendHolidays,WeekdayHolidays,GDPGrowth%,MoMInflation%,WoWUSDChange%,Litres(w-1),Litres(w-2),Litres(w-3),Litres(w-49),Litres(w-50),Litres(w-51),Litres(w-52),WeekNumber,Benzin,Motorin,Akdeniz Bölgesi,Doğu Anadolu Bölgesi,Ege Bölgesi,Güneydoğu Anadolu Bölgesi,Karadeniz Bölgesi,Marmara Bölgesi,İç Anadolu Bölgesi,Autumn,Fall,Summer,Winter
0,2015-01-05,4215.0,-0.961919,-0.867481,-0.258041,-0.296251,-0.403232,-0.743528,-0.958067,-0.534953,-0.629324,-0.423602,-0.747305,-0.521297,-0.542346,-0.769578,-1.671497,1.035544,-1.035544,-0.401073,-0.185584,-0.471881,-0.233197,3.146363,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
1,2015-01-05,18791.0,-1.432183,-0.867481,-0.258041,-0.296251,-0.403232,-0.743528,-0.958067,0.075114,-0.011383,0.175361,-0.072688,0.101894,0.426865,-0.193721,-1.671497,-0.965676,0.965676,-0.401073,-0.185584,-0.471881,-0.233197,3.146363,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
2,2015-01-05,23570.0,-1.432183,-0.867481,-0.258041,-0.296251,-0.403232,-0.743528,-0.958067,-0.271255,-0.516920,-0.754462,-0.860097,-0.863684,-0.038296,-0.155044,-1.671497,-0.965676,0.965676,-0.401073,-0.185584,-0.471881,-0.233197,3.146363,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
3,2015-01-05,3609.0,-0.961919,-0.867481,-0.258041,-0.296251,-0.403232,-0.743528,-0.958067,-0.745788,-0.668894,-0.754462,-0.860097,-0.755481,-0.864936,-0.868177,-1.671497,1.035544,-1.035544,-0.401073,-0.185584,-0.471881,-0.233197,3.146363,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
4,2015-01-05,21576.0,-1.432183,-0.867481,-0.258041,-0.296251,-0.403232,-0.743528,-0.958067,-0.745788,-0.249907,-0.754462,-0.860097,-0.212537,-0.864936,-0.286881,-1.671497,-0.965676,0.965676,-0.401073,-0.185584,-0.471881,-0.233197,3.146363,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492744,2019-12-02,NaN,1.763225,-1.031720,-0.258041,-0.296251,-1.299962,1.004695,0.049522,-0.745788,-0.752597,-0.754462,-0.860097,-0.863684,-0.864936,-0.868177,1.489562,1.035544,-1.035544,-0.401073,5.388384,-0.471881,-0.233197,-0.317827,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
492745,2019-12-09,NaN,1.763225,-1.031720,-0.258041,-0.296251,-0.627415,1.004695,0.061059,-0.745788,-0.752597,-0.754462,-0.860097,-0.863684,-0.864936,-0.868177,1.556818,1.035544,-1.035544,-0.401073,5.388384,-0.471881,-0.233197,-0.317827,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
492746,2019-12-16,NaN,1.763225,-1.031720,-0.258041,-0.296251,-0.627415,-0.551416,0.699455,-0.745788,-0.752597,-0.754462,-0.860097,-0.863684,-0.864936,-0.868177,1.624075,1.035544,-1.035544,-0.401073,5.388384,-0.471881,-0.233197,-0.317827,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317
492747,2019-12-23,NaN,1.763225,-1.031720,-0.258041,-0.296251,-0.627415,-0.551416,-0.062005,-0.745788,-0.752597,-0.754462,-0.860097,-0.863684,-0.864936,-0.868177,1.691331,1.035544,-1.035544,-0.401073,5.388384,-0.471881,-0.233197,-0.317827,-0.718832,-0.439697,-0.594538,-0.579812,-0.575553,1.787317


Separate TrainingAndTestData

In [36]:
df_training = df[df.WeekStarting < '2018-12-31']
df_test = df[df.WeekStarting > '2018-12-31']

Save data

In [40]:
df_training.to_pickle('TrainingDataFinalized.pkl')
df_test.to_pickle('TestDataFinalized.pkl')
